In [ ]:
#Import and define function
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
!git clone https://github.com/Louismac/NLP-Public
%cd NLP-Public

def show_audio(y, sr=44100, normalise=True):
    fig, ax = plt.subplots(figsize=(16,4))
    ax.set_ylim(-1,1)
    librosa.display.waveplot(y, sr=sr, ax=ax)
    return ipd.Audio(y, rate=sr, autoplay=True)

# Audio Task 

### Task 1

Have a go at loading in a chopping up the drum tracks. You can also try your own material (see links for free samples below)!

- Take one of the drum breaks from the demos and see if you can play around with. Can you 
    - Take part of it and put it somewhere else in the file?
    - Take take of it, alter it (change speed with `librosa.resample`, volume, reverse with `np.flip`), and put it somewhere else in the file
- Can you chain some of these actions together to chop up the drum beat?
- Can you start including audio from different samples? Remember, one of the reasons the Jungle algorithm works in the lectures work is because they're all the same tempo and length. 
    - Check out [FreeSound](https://freesound.org/) for some free samples or these [BBC 1xtra](https://www.bbc.co.uk/1xtra/1xmusic/sample/) samples, or some audio samples from an [2014 MIR workshop](https://ccrma.stanford.edu/workshops/mir2014/audio/)
    

In [ ]:
#Load in your own audio files (or use some of our drums loops (check the audio/ folder))
y, sr = librosa.load('audio/soul_piano.wav')
show_audio(y, sr)

In [ ]:
### Edit and chop up here!


### Task 2
Have a play with Louis' Jungle Algorithm. It should be different every time you run it. 
- Try some different `chop` values (1/2, 1/4, 1/16, 1/32), how does that effect the music?
- Every time you see `if np.random.rand() > [NUMBER]:`, there is a probability effecting an outcome, try some different numbers between 0 and 1 and see what happens 
- Towards the bottom there is a line which adds in some vocals
   ```sig[chan,start:end] = sig[chan][start:end] + vox```
    -  Can you change it so that it overwrites instead of mixes with the drum backing track?
    -  Can you add in some more samples (doesn't have to be vocals)
- Can you add some more code at the end so that the guitar plays throughout the whole track?
- **BONUS** You can try swapping out the drum samples. This will work for any array of samples you load in as long they are the same length (and ideally at the tempo).

In [ ]:
#Load in 8 drum breaks
multi_drums = np.array([librosa.load('audio/drums' + str(i) +'.wav')[0] for i in range(8)])
print(multi_drums.shape)
#Load in 2 drum vocal samples
vocals = np.array([librosa.load('audio/vocal' + str(i) +'.wav')[0] for i in range(2)])
guitar, sr = librosa.load('audio/guitar.wav')

In [ ]:
bar = len(multi_drums[0])/2
beat = int(bar / 4)
sq = int(beat / 4)
track_length = int(bar * 8)
#Start with 2 channels of silence (0's)
sig = np.array([np.zeros(track_length), np.zeros(track_length)])

####CHANGING CHOP LENGTH########
#Length of each chop (smaller number is more frequent chops)
chop = 1/8
chop_size = int(len(multi_drums[0]) * chop)
n = int(track_length/chop_size)
print("making track with " + str(n) + " sections")
#Pick a random drum break and location
drum_index = np.random.randint(len(multi_drums))
chop_at = np.random.randint(1/chop)*chop_size
##Chop together drum tracks
for i in range(n):
    
    #65% chance of picking new drums
    if np.random.rand() > 0.35:
        #Pick a random drum break and location
        drum_index = np.random.randint(len(multi_drums))
        chop_at = np.random.randint(1/chop)*chop_size
        
    #Get a section of that drum break
    new_drums = multi_drums[drum_index, chop_at:chop_at+chop_size]
    
    #10% chance of reversing
    if np.random.rand() > 0.9:
        print("reversing")
        new_drums = np.flip(new_drums)
        
    #10% chance of speeding up
    if np.random.rand() > 0.9:
        #twice the pitch, half the length
        print("speed up!")
        new_drums = librosa.resample(new_drums, sr, sr/2)
        
    #10% chance of slowing down up
    if np.random.rand() > 0.9:
        #half the pitch, twice the length
        print("sloooow")
        new_drums = librosa.resample(new_drums, sr, sr*2)
        new_drums = new_drums[0:chop_size]
        
    #Pick channel
    chan = np.random.randint(2)
    #Overwrite with new drums
    start = i*chop_size
    end = start + len(new_drums)
    print("left" if chan == 0 else "right")
    print("adding " + str(len(new_drums)) + " samples from " + str(start) + " to " + str(end))
    sig[chan,start:end] = new_drums

##Add vocals    
for i in range(n):
    #10% chance of adding in vocal sample
    if np.random.rand() > 0.9:
        #Pick a sample
        vox = vocals[np.random.randint(len(vocals))]
        start = i*chop_size
        end = start+len(vox)
        #Make sure we dont go off the end
        if end < len(sig[0]):
            #Pick channel
            chan = np.random.randint(2)
            #Adding (not overwrite)
            sig[chan,start:end] = sig[chan][start:end] + vox

#Add the guitar at the end (not overwrite)
#Slice the audio from the end back to the length of the guitar, then add guitar audio and put back
#We add to both channels using the : and broadcasting (more on that next week)
sig[:,-len(guitar):] = sig[:,-len(guitar):] + guitar    
show_audio(sig, sr)

## Analysing music 

### Task 3

Try some of your own music in the beat tracker. What genres does it work for? What genres does it struggle with? Place your answers in the forum

If you don't music files on your computer, there are places to find royalty free music online e.g. [Blue Dot Sesssions](https://www.sessions.blue/) or [Free Music Archive](https://freemusicarchive.org/curator/Creative_Commons)

You can also try [Epidemic Sounds](https://player.epidemicsound.com/students/) as a student and get free music as long as you're not publishing it anywhere.


In [ ]:
### Add in your own audio files 
y, sr = librosa.load('audio/drums2.wav')
tempo, beats = librosa.beat.beat_track(y=y, sr=sr)
y_beats = librosa.clicks(frames=beats, sr=sr, length = len(y))
show_audio(y + y_beats, sr)


### Task 4

Think about this for the discussion at the end, what are some qualities of sound or music that you think machines would have trouble analysing? Place your answers in the forum